<a href="https://colab.research.google.com/github/DipeshDhote/MULTIPLE_SOURCE_RAG_NOTEBOOK/blob/main/Agent_multi_source.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install arxiv
! pip install langchain_community

In [2]:
!pip install wikipedia

In [3]:
! pip install pypdf

In [4]:
! pip install faiss-gpu

In [31]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [32]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [33]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader= PyPDFLoader("attention.pdf")
docs=loader.load()


In [8]:
! pip install langchain_google_genai

In [34]:
from google.colab import userdata
import os

In [35]:
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

In [36]:
documents = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=80).split_documents(docs)


from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vectordb=FAISS.from_documents(documents,embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x785bccd0ece0>)

In [37]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [38]:
retriever_tool.name

'langsmith_search'

In [39]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [40]:
tools=[wiki,arxiv,retriever_tool]

In [41]:
groq_api_key = userdata.get("GROQ_API")


In [17]:
! pip install langchain_groq

In [42]:
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=0.5, groq_api_key=groq_api_key, model_name = "llama3-8b-8192")

In [19]:
! pip install langchainhub

In [43]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [44]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [45]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [47]:
response = agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
allowed moves of arbitrary length. We show that the diameter of these graphs on
fibers of a fixed intege
Invoking: `wikipedia` with `{'query': 'Heat-bath random walks with Markov bases'}`


Page: List of atheists in science and technology
Summary: This is a list of atheists in science and technology. A statement by a living person that he or she does not believe in God is not a sufficien
Invoking: `arxiv` with `{'query': 'Heat-bath random walks with Markov bases'}`


Published: 2017-12-16
Title: Probability distributions for Markov chains based quantum walks
Authors: Radhakrishnan Balu, Chaobin Liu, Salvador E. Venegas-Andraca
Summary: We analyze the probability distributions of the quantum walks induce

{'input': "What's the paper 1605.08386 about?",
 'output': 'No more tool calls.'}

In [ ]:
print(response['output'])